In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

### Scrape billboard hot 100

In [112]:
billboard_top_100 = pd.DataFrame([], columns=['chart_rank', 'artist', 'song_title'])

In [113]:
def scrape_billboard_hot_100(dataset):
    r = requests.get('https://www.billboard.com/charts/hot-100/')
    soup = BeautifulSoup(r.content, 'html.parser')
    song_rows = soup.find_all('div', attrs={'class': 'o-chart-results-list-row-container'})
    for i, row in enumerate(song_rows):
        song = row.find('h3', attrs={'class': 'a-no-trucate', 'id': 'title-of-a-story'})    
        artist =  row.find_all('span', attrs={'class': 'c-label'})[1]

        dataset.at[dataset.shape[0]] = [i+1, artist.get_text(strip=True), 
                                        song.get_text(strip=True)]

In [114]:
scrape_billboard_hot_100(billboard_top_100)

In [115]:
billboard_top_100

,chart_rank,artist,song_title
0,1,Miley Cyrus,Flowers
1,2,SZA,Kill Bill
2,3,"Metro Boomin, The Weeknd & 21 Savage",Creepin'
3,4,PinkPantheress & Ice Spice,"Boy's A Liar, Pt. 2"
4,5,Morgan Wallen,Last Night
...,...,...,...
95,96,Jordan Davis,Next Thing You Know
96,97,Metro Boomin Featuring Don Toliver & Future,Too Many Nights
97,98,Nengo Flow & Bad Bunny,Gato de Noche
98,99,NEW,Painting Pictures


In [117]:
billboard_top_100.to_csv('billboard_100.csv', index=False)

### Expand collection of songs and artists

- Scrape page www.discogs.com
- get items for five music genres: Electronic, Pop, Jazz, Rock, HipHop
- scrape only the first 10 pages (max) with up to 250 items per page 

In [107]:
HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
GENRES = ['Electronic', 'Pop', 'Jazz', 'Rock', 'Hip+Hop']

In [104]:
music_database = pd.DataFrame([], columns=['gerne', 'artist', 'song_title'])

In [103]:
def extract_discogs_list_info(row):
    artist = row.find_all('a')[0].get_text(strip=True)
    song = row.find_all('a')[1].get_text(strip=True)
    return artist, song


def extract_discogs_page_info(url, dataset, genre):
    res = requests.get(url_music, headers=HEADERS)
    soup = BeautifulSoup(res.content, 'html.parser')
    discogs_row = soup.find_all('li', attrs={'role': 'listitem'})
    
    for row in discogs_row:
        dataset.at[dataset.shape[0]] = [genre, *extract_discogs_list_info(row)]

def scrape_discogs_website(list_genres):
    for genre in list_genres:
        for i in range(1, 11):
            url_music = f'https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&ev=gs_mc&layout=sm&genre_exact={genre}&page={i}'
            extract_discogs_page_info(url_music, music_database, genre)

In [108]:
scrape_discogs_website(GENRES)

In [109]:
music_database

,gerne,artist,song_title
0,Electronic,Daft Punk,Discovery
1,Electronic,Daft Punk,Homework
2,Electronic,Björk,Debut
3,Electronic,Lady Gaga,Chromatica
4,Electronic,Madonna,Ray Of Light
...,...,...,...
12445,Hip+Hop,C + C Music Factory,Gonna Make You Sweat
12446,Hip+Hop,Daft Punk,TRON: Legacy (Vinyl Edition Motion Picture Sou...
12447,Hip+Hop,Madonna,Keep It Together
12448,Hip+Hop,Mousse T.,Hot 'N' Juicy


In [110]:
music_database.to_csv('music_discogs.csv', index=False)